In [1]:
import re,string
import math, re, string, requests, json
from itertools import product
from inspect import getsourcefile
from os.path import abspath, join, dirname
import nbimporter
from slistener import SListener
import time, tweepy, sys
import pandas as pd
import json
from pandas.io.json import json_normalize

Importing Jupyter notebook from slistener.ipynb


In [2]:
REMOVE_PUNCTUATION = re.compile('[%s]' % re.escape(string.punctuation))

PUNCTUATION_LIST = [".", "!", "?", ",", ";", ":", "-", "'", "\"",
             "!!", "!!!", "??", "???", "?!?", "!?!", "?!?!", "!?!?"]

NEGATION_WORDS = ["aint", "arent", "cannot", "cant", "couldnt", "darent", "didnt", "doesnt",
 "ain't", "aren't", "can't", "couldn't", "daren't", "didn't", "doesn't",
 "dont", "hadnt", "hasnt", "havent", "isnt", "mightnt", "mustnt", "neither","no",
 "don't", "hadn't", "hasn't", "haven't", "isn't", "mightn't", "mustn't",
 "neednt", "needn't", "never", "none", "nope", "nor", "not", "nothing", "nowhere",
 "oughtnt", "shant", "shouldnt", "uhuh", "wasnt", "werent",
 "oughtn't", "shan't", "shouldn't", "uh-uh", "wasn't", "weren't",
 "without", "wont", "wouldnt", "won't", "wouldn't", "rarely", "seldom", "despite"]

CONTRASTING_CONJUNCTION = \
{"but", "although", "though", "even though", "even if", "however"}

# Sentiment Incremental score for capitalization approach.
CAPS_INCR = 1.00

# Sentiment score for Punctuation Emphsis symbols
QUESTION_INCREASE = 0.1
EXCLAMATION_INCREASE = 0.25
NEGATION_MODIFIER = -1.00

# Score for Degree Modifiers

SCORE_INCREASE = 0.5
SCORE_DECREASE = -0.5


DEGREE_DICTIONARY = \
    {"100 percent": SCORE_INCREASE, "a good deal": SCORE_INCREASE, "a great deal": SCORE_INCREASE,
     "a lot": SCORE_INCREASE,
     "aboundingly": SCORE_INCREASE, "absolutely": SCORE_INCREASE, "absurdly": SCORE_INCREASE,
     "abundantly": SCORE_INCREASE,
     "admirably": SCORE_INCREASE, "alarmingly": SCORE_INCREASE, "amazingly": SCORE_INCREASE,
     "astronomically": SCORE_INCREASE, "awfully": SCORE_INCREASE, "breathtakingly": SCORE_INCREASE,
     "clearly": SCORE_INCREASE, "completely": SCORE_INCREASE, "considerably": SCORE_INCREASE, "crazy": SCORE_INCREASE,
     "damn": SCORE_INCREASE, "damned": SCORE_INCREASE, "darn": SCORE_INCREASE, "darned": SCORE_INCREASE,
     "decidedly": SCORE_INCREASE, "deeply": SCORE_INCREASE, "deservedly": SCORE_INCREASE, "downright": SCORE_INCREASE,
     "dreadfully": SCORE_INCREASE,  # up to here so far
     "effing": SCORE_INCREASE, "enormously": SCORE_INCREASE,
     "entirely": SCORE_INCREASE, "especially": SCORE_INCREASE, "exceptionally": SCORE_INCREASE,
     "extremely": SCORE_INCREASE,
     "fabulously": SCORE_INCREASE, "flipping": SCORE_INCREASE, "flippin": SCORE_INCREASE,
     "fricking": SCORE_INCREASE, "frickin": SCORE_INCREASE, "frigging": SCORE_INCREASE, "friggin": SCORE_INCREASE,
     "fully": SCORE_INCREASE, "fucking": SCORE_INCREASE,
     "greatly": SCORE_INCREASE, "hella": SCORE_INCREASE, "highly": SCORE_INCREASE, "hugely": SCORE_INCREASE,
     "incredibly": SCORE_INCREASE,
     "intensely": SCORE_INCREASE, "majorly": SCORE_INCREASE, "more": SCORE_INCREASE, "most": SCORE_INCREASE,
     "particularly": SCORE_INCREASE,
     "purely": SCORE_INCREASE, "quite": SCORE_INCREASE, "really": SCORE_INCREASE, "remarkably": SCORE_INCREASE,
     "so": SCORE_INCREASE, "substantially": SCORE_INCREASE,
     "thoroughly": SCORE_INCREASE, "totally": SCORE_INCREASE, "tremendously": SCORE_INCREASE,
     "uber": SCORE_INCREASE, "unbelievably": SCORE_INCREASE, "unusually": SCORE_INCREASE, "utterly": SCORE_INCREASE,
     "very": SCORE_INCREASE,
     "almost": SCORE_DECREASE, "barely": SCORE_DECREASE, "hardly": SCORE_DECREASE, "just enough": SCORE_DECREASE,
     
     "less": SCORE_DECREASE, "little": SCORE_DECREASE, "marginally": SCORE_DECREASE, "occasionally": SCORE_DECREASE,
     "partly": SCORE_DECREASE,
     "scarcely": SCORE_DECREASE, "slightly": SCORE_DECREASE, "somewhat": SCORE_DECREASE,
     "sort of": SCORE_DECREASE, "sorta": SCORE_DECREASE, "sortof": SCORE_DECREASE, "sort-of": SCORE_DECREASE}

In [3]:
"""
    Check if there is negation contect in the sentence or not.
    Still continuing
"""
def negated(input_words, include_nt=True):
    
    neg_words = []
    neg_words.extend(NEGATION_WORDS)
      
    for word in neg_words:
        if word in input_words:
            #print(word)
            return True
    if include_nt:
        for word in input_words:
            #print(word)
            if "n't" in word:
                return True
       
    return False

def capitalized(words):
        
        emphasis = False
        capitalized_words = 0

        for word in words:
            if word.isupper():
                capitalized_words += 1
        cap_differential = len(words) - capitalized_words
        if 0 < cap_differential < len(words):
            emphasis = True

        return emphasis


def alter_valence(word, valence, is_cap_diff):
   
    modifier = 0.0
    word_lower = word.lower()
    if word_lower in DEGREE_DICTIONARY:
        modifier = DEGREE_DICTIONARY[word_lower]
        if valence < 0:
            modifier = modifier * -1
                    
        if word.isupper() and is_cap_diff:
            if valence > 0:
                modifier += CAPS_INCR
            else: modifier -= CAPS_INCR
    return modifier

In [5]:
class SentimentAnalyzer(object):
    
    def __init__(self, lexicon_file="New_Lexicon.txt"):
        _this_module_file_path_ = abspath(getsourcefile(lambda:0))
        lexicon_full_filepath = join(dirname(_this_module_file_path_), lexicon_file)
        with open(lexicon_full_filepath) as f:
            self.lexicon_full_filepath = f.read()
        self.lexicon = self.make_lex_dict()
    
    def make_lex_dict(self):
        
        lex_dict = {}
        for line in self.lexicon_full_filepath.split('\n'):
            (word, measure) = line.strip().split('\t')[0:2]
            lex_dict[word] = float(measure)
            
        return lex_dict
       

    def polarity_scores(self, text):
        
        sentimenttext = SentimentText(text)
        sentiments = []
        words_and_emoticons = sentimenttext.words_and_emoticons
        
        for item in words_and_emoticons:
            valence = 0
            i = words_and_emoticons.index(item)
            if (i < len(words_and_emoticons) - 1 and item.lower() == "kind" and
                        words_and_emoticons[i + 1].lower() == "of") and item.lower() in DEGREE_DICTIONARY:
                sentiments.append(valence)
            
                continue

            sentiments = self.sentiment_score(valence, sentimenttext, item, i, sentiments)
        sentiments = self._contains_but(words_and_emoticons, sentiments)
        
        
        return(sentiments)
    
    def _punctuation_emp(self, sum_s, text):
       
        ep_amplifier = self._exclamation_score(text)
        qm_amplifier = self._questionmark_score(text)
        punct_emph_amplifier = ep_amplifier+qm_amplifier
        return punct_emph_amplifier

    def _exclamation_score(self, text):
        
        ep_count = text.count("!")
        if ep_count > 4:
            ep_count = 4
        
        ep_amplifier = ep_count * EXCLAMATION_INCREASE
        return ep_amplifier

    def _questionmark_score(self, text):
        
        qm_count = text.count("?")
        qm_amplifier = 0
        if qm_count > 1:
            if qm_count <= 3:
                qm_amplifier = qm_count * QUESTION_INCREASE
            else:
                qm_amplifier = 1.00
        return qm_amplifier
    
    
    def _contains_negation(self, valence, words_and_emoticons, start_i, i):
        
        if start_i == 0:
            if negated([words_and_emoticons[i-1]]):
                    valence = valence*NEGATION_MODIFIER
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
        if start_i == 1:
            if words_and_emoticons[i-2] == "never" and\
               (words_and_emoticons[i-1] == "so" or
                words_and_emoticons[i-1] == "this"):
                valence = valence*1.5
            elif negated([words_and_emoticons[i-(start_i+1)]]):
                valence = valence*NEGATION_MODIFIER
        if start_i == 2:
            if words_and_emoticons[i-3] == "never" and \
               (words_and_emoticons[i-2] == "so" or words_and_emoticons[i-2] == "this") or \
               (words_and_emoticons[i-1] == "so" or words_and_emoticons[i-1] == "this"):
                valence = valence*1.25
            elif negated([words_and_emoticons[i-(start_i+1)]]):
                valence = valence*NEGATION_MODIFIER

        
        return valence
        
        
    def sentiment_score(self, valence, sentitext, item, i, sentiments):

        is_cap_diff = sentitext.is_cap_diff
        words_and_emoticons = sentitext.words_and_emoticons
        item_lowercase = item.lower()
        if item_lowercase in self.lexicon:
            valence = self.lexicon[item_lowercase]
            
            if item.isupper() and is_cap_diff:
                if valence > 0:
                    valence += CAPS_INCR
                else:
                    valence -= CAPS_INCR

            
            for start_i in range(0,4):
                if i > start_i and words_and_emoticons[i-(start_i+1)].lower() not in self.lexicon:
                    s = alter_valence(words_and_emoticons[i-(start_i+1)], valence, is_cap_diff)
                    valence = valence+s
                    valence = self._contains_negation(valence, words_and_emoticons, start_i, i)
        
        sentiments.append(valence)
        return sentiments
                     
    
    """
    Algorithm is defined as such:
    The contextual conjective "but" clause works in way to calculate the valence score , algorithm states that the there is 50% 
    reduction in valence score before "but" clause words and 150% increase in the valence score after the "but" clause
    sentence.

    """ 
    
    def _contains_but(self,words_and_emoticons, sentiments):
       
        CONTRASTING_CONJUNCTION = \
        {"but", "although", "though", "even though", "even if", "however"}
        
        for conj_word in CONTRASTING_CONJUNCTION:
            if conj_word in words_and_emoticons:
                but_index = words_and_emoticons.index(conj_word)
                for sentiment in sentiments:
                    sentiment_index = sentiments.index(sentiment)
                    
                    if sentiment_index < but_index:
                        sentiments.pop(sentiment_index)
                        sentiments.insert(sentiment_index, sentiment * 0.5)
                    
                    elif sentiment_index > but_index:
                        sentiments.pop(sentiment_index)
                        sentiments.insert(sentiment_index, sentiment * 1.5)   
                        
        return sentiments          
        
  

In [6]:
class SentimentText(object):
    
    def __init__(self, text):
        if not isinstance(text, str):
            text = str(text.encode('utf-8'))
        self.text = text
        self.words_and_emoticons = self._words_and_emoticons()
        self.is_cap_diff = capitalized(self.words_and_emoticons)
        
    def _words_and_punc(self):
        
        no_punc_text = REMOVE_PUNCTUATION.sub('', self.text)
        words_only = no_punc_text.split()
        words_only = set( w for w in words_only if len(w) > 1 )
        punc_before = {''.join(p): p[1] for p in product(PUNCTUATION_LIST, words_only)}
        punc_after = {''.join(p): p[0] for p in product(words_only, PUNCTUATION_LIST)}
        words_punc_dict = punc_before
        words_punc_dict.update(punc_after)
        return words_punc_dict

    def _words_and_emoticons(self):
        
        wes = self.text.split()
        words_punc_dict = self._words_and_punc()       
        wes = [we for we in wes if len(we) > 1]
        for i, we in enumerate(wes):
            if we in words_punc_dict:
                wes[i] = words_punc_dict[we]
        return wes

In [7]:
if __name__ == '__main__':
    
    analyzer = SentimentAnalyzer()
    Idioms_data = pd.read_csv("Idioms_File.csv")
    text_sentence = Idioms_data['definition']
    def make_list():
        result = []
        for sentence in text_sentence:
            vs = analyzer.polarity_scores(sentence)
            total_score = format(float(sum(vs)),'.2f')
            result.append(total_score)
        return result

In [10]:
temp = make_list()
Idioms_data['Sentiment_Score'] = temp
Idioms_data.to_csv("Idioms_Sentiment_Score.csv")